In [66]:
import h2o
import pandas
import matplotlib.pyplot as plt

In [67]:
# Connect to a cluster
h2o.init()

H2O cluster uptime:,5 seconds 520 milliseconds
H2O cluster version:,3.1.0.99999
H2O cluster name:,anqi_fu
H2O cluster total nodes:,1
H2O cluster total memory:,3.56 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [69]:
# air_path = [h2o.locate("bigdata/laptop/airlines_all.05p.csv")]
# air_path = [h2o.locate("bigdata/laptop/flights-nyc/flights14.csv.zip")]
air_path = [h2o.locate("smalldata/airlines/allyears2k_headers.csv")]

# ----------

# 1- Load data - 1 row per flight.  Has columns showing the origin,
# destination, departure and arrival time, carrier information, and
# whether the flight was delayed.
print "Import and Parse airlines data"
data = h2o.import_frame(path=air_path)
data.describe()

Import and Parse airlines data

Parse Progress: [##################################################] 100%

Parsed 43,978 rows and 31 cols:



File1,/Users/anqi_fu/Documents/workspace/h2o-dev/smalldata/airlines/allyears2k_headers.csv


Rows: 43,978 Cols: 31

Chunk compression summary:



chunk_type,chunk_name,count,count_percentage,size,size_percentage
C0L,Constant Integers,116,11.0056925,9.1 KB,0.65225154
C0D,Constant Reals,171,16.22391,13.4 KB,0.9615087
CBS,Bits,10,0.94876665,2.2 KB,0.1600406
CX0,Sparse Bits,53,5.0284634,5.0 KB,0.35733265
CXI,Sparse Integers,6,0.56926,752 B,0.052854866
C1,1-Byte Integers,174,16.50854,237.4 KB,17.087515
C1N,1-Byte Integers (w/o NAs),131,12.428843,172.1 KB,12.389265
C1S,1-Byte Fractions,63,5.9772296,83.7 KB,6.02686
C2,2-Byte Integers,330,31.309298,865.8 KB,62.31237



Frame distribution summary:



,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.16:54321,1.4 MB,43978.0,34.0,1054.0
mean,1.4 MB,43978.0,34.0,1054.0
min,1.4 MB,43978.0,34.0,1054.0
max,1.4 MB,43978.0,34.0,1054.0
stddev,0 B,0.0,0.0,0.0
total,1.4 MB,43978.0,34.0,1054.0



Column-by-Column Summary:



,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
type,int,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,int,enum,enum,int,int,int,int,enum,int,int,int,int,int,int,enum,enum
mins,1987.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,16.0,17.0,14.0,-63.0,-16.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1997.5,1.40909090909,14.6010732639,3.82061485288,1345.84666138,1313.22286143,1504.63413038,1485.28916731,6.93549047251,818.842989677,2376.09750603,124.814529135,125.021562607,114.316111091,9.31711193698,10.0073906556,61.1830005912,78.5525944791,730.182190565,5.38136805953,14.1686341847,0.0246941652645,2.98608349901,0.00247851198326,4.04780029106,0.289376469271,4.85503190418,0.0170155602821,7.62006045002,0.555755150302,0.525057983537
maxs,2008.0,10.0,31.0,7.0,2400.0,2359.0,2400.0,2359.0,9.0,3949.0,3500.0,475.0,437.0,402.0,475.0,473.0,131.0,133.0,3365.0,128.0,254.0,1.0,3.0,1.0,369.0,201.0,323.0,14.0,373.0,1.0,1.0
sigma,6.34436090171,1.87471137134,9.17579042586,1.90501311913,465.340899124,476.251139993,484.347487904,492.750434123,2.05121227084,777.404369164,1168.75931155,73.9744416606,73.40159463,69.6363295151,29.8402219624,26.4388090429,37.64411521,33.930070329,578.43800823,4.20197993986,9.9050857472,0.155193141358,0.182676305421,0.0497234872189,16.2057299045,4.41677989873,18.6197762215,0.403940182102,23.4875658741,0.496887288343,0.499377380318
zero_count,0,0,0,0,0,569,0,569,724,0,2,0,0,0,1514,6393,59,172,0,623,557,42892,81,43869,7344,8840,7388,8914,7140,19537,20887
missing_count,0,0,0,0,1086,0,1195,0,0,0,32,1195,13,16649,1195,1086,0,0,35,16026,16024,0,9774,0,35045,35045,35045,35045,35045,0,0


In [70]:
# ----------

# 2- Data exploration and munging. Generate scatter plots 
# of various columns and plot fitted GLM model.

# Function to fit a GLM model and plot the fitted (x,y) values
def scatter_plot(data, x, y, max_points = 1000, fit = True):
    if(fit):
        lr = h2o.glm(x = data[[x]], y = data[y], family = "gaussian")
        coeff = lr.coef()
    df = data[[x,y]]
    runif = df[y].runif()
    df_subset = df[runif < float(max_points)/data.nrow()]
    df_py = h2o.as_list(df_subset)
    df_py = zip(*df_py)    # Convert list of rows into list of columns
    
    if(fit): h2o.remove(lr._key)
    
    plt.plot(df_py[0], df_py[1], "yo")
    plt.xlabel(x)
    plt.ylabel(y)
    
    if(fit):
        x_min = min(df_py[0])
        x_max = max(df_py[0])
        y_min = coeff["Intercept"] + coeff[x]*x_min
        y_max = coeff["Intercept"] + coeff[x]*x_max
        plt.plot([x_min, x_max], [y_min, y_max], "k-")
    plt.show()

scatter_plot(data, "Distance", "AirTime", fit = True)
scatter_plot(data, "UniqueCarrier", "ArrDelay", max_points = 5000, fit = False)


glm Model Build Progress: [##################################################] 100%


In [55]:
# Group flights by month
aggregates = {"Month": ["nrow", 0, "all"], "Cancelled": ["sum", 1, "all"]}
bpd = h2o.group_by(data, cols=["Month"], aggregates=aggregates)
bpd.show()
bpd.describe()
bpd.dim()

# Convert columns to factors
data["Year"]      = data["Year"]     .asfactor()
data["Month"]     = data["Month"]    .asfactor()
data["DayOfWeek"] = data["DayOfWeek"].asfactor()
data["Cancelled"] = data["Cancelled"].asfactor()

EnvironmentError: h2o-py got an unexpected HTTP status code:
 412 Precondition Failed (method = POST; url = http://localhost:54321/3/Rapids). 
detailed error messages: java.lang.NullPointerException

In [62]:
# Calculate and plot travel time
hour1 = data["CRSArrTime"] / 100
mins1 = data["CRSArrTime"] % 100
arrTime = hour1*60 + mins1

hour2 = data["CRSDepTime"] / 100
mins2 = data["CRSDepTime"] % 100
depTime = hour2*60 + mins2

# TODO: If-else method does not work in Python
travelTime = [x if x > 0 else None for x in (arrTime - depTime)]
data["TravelTime"] = travelTime
scatter_plot(data, "Distance", "TravelTime")

TypeError: unsupported operand type(s) for %: 'instance' and 'int'

In [71]:
# Impute missing travel times and re-plot
h2o.impute(data = data, column = "Distance", by = ["Origin", "Dest"])
scatter_plot(data, "Distance", "TravelTime")

ValueError: Name TravelTime not in Frame

In [72]:
# ----------
# 3- Fit a model on train; using test as validation

# Create test/train split
data_split = h2o.split_frame(data, ratios = [0.75,0.25])
train = data_split[1]
test  = data_split[2]

# Set predictor and response variables
myY = "IsDepDelayed"
myX = ["Origin", "Dest", "Year", "UniqueCarrier", "DayOfWeek", "Month", "Distance", "FlightNum"]

# Simple GLM - Predict Delays
data_glm = h2o.glm(x           =train[myX],
                   y           =train[myY],
                   validation_x=test [myX],
                   validation_y=test [myY],
                   family      ="binomial",
                   standardize =True)

# Simple GBM
data_gbm = h2o.gbm(x              =train[myX],
                   y              =train[myY],
                   validation_x   =test [myX],
                   validation_y   =test [myY],
                   balance_classes=True,
                   ntrees         =3,
                   max_depth      =1,
                   distribution   ="bernoulli",
                   learn_rate     =0.1,
                   min_rows       =2)

# Complex GBM
data_gbm2 = h2o.gbm(x              =train[myX],
                    y              =train[myY],
                    validation_x   =test [myX],
                    validation_y   =test [myY],
                    balance_classes=True,
                    ntrees         =50,
                    max_depth      =5,
                    distribution   ="bernoulli",
                    learn_rate     =0.1,
                    min_rows       =2)

# Simple Random Forest
data_rf = h2o.random_forest(x              =train[myX],
                            y              =train[myY],
                            validation_x   =test [myX],
                            validation_y   =test [myY],
                            ntrees         =5,
                            max_depth      =2,
                            balance_classes=True)

# Complex Random Forest
data_rf2 = h2o.random_forest(x              =train[myX], 
                             y              =train[myY],
                             validation_x   =test [myX],
                             validation_y   =test [myY],
                             ntrees         =10,
                             max_depth      =5,
                             balance_classes=True)

# Deep Learning with 5 epochs
data_dl = h2o.deeplearning(x                   =train[myX],
                           y                   =train[myY],
                           validation_x        =test [myX],
                           validation_y        =test [myY],
                           hidden              =[10,10],
                           epochs              =5,
                           variable_importances=True,
                           balance_classes     =True,
                           loss                ="Automatic")

EnvironmentError: h2o-py got an unexpected HTTP status code:
 404 Not Found (method = GET; url = http://localhost:54321/3/Frames/py559f0dcf-2cbf-4b14-953a-f5c30eb13b49_part0). 
detailed error messages: Object 'py559f0dcf-2cbf-4b14-953a-f5c30eb13b49_part0' not found for argument: key

In [65]:
# Variable importances from each algorithm
abs(data_glm.coef_norm())   # Magnitude of normalized coefficients
data_gbm.varimp()
data_rf.varimp()

NameError: name 'data_glm' is not defined

In [17]:
# Predict on test dataset with GBM model
best_pred = predict(data_gbm2, test)

NameError: name 'predict' is not defined